# **FINANCE 361**

**Lecture 2**

## **Statistical Regression vs Neural Networks**

(The following Python code is non-examinable)

This notebook is used to facilitate discussion on statistical regressions and neural networks.


*   The data utilized in the analysis does not meet the highest quality standards.
*   The coding approach does not align with the industry's best practices.
*   The code is designed to be easily understood and intuitive.



### **Preliminary Step #1**


*   Install the yfinance Python package
  *   To download market data from Yahoo! Finance API
  *   See: https://pypi.org/project/yfinance/

*   Install the Keras Python package
  *   A Python interface for artificial neural networks
  *   See: https://keras.io/

In [ ]:
pip install yfinance

In [ ]:
pip install --upgrade keras

### **Preliminary Step #2**



*   Import the required packages into Python for our models
  * Numpy:  https://numpy.org
  * Pandas: https://pandas.pydata.org
  * Matplotlib: https://matplotlib.org
  * Statsmodels: https://www.statsmodels.org/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

import yfinance as yf

import statsmodels.api as sm

import keras

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K

print(keras.__version__)

np.random.seed(0)

### **Step 0: Define the parameters**

Defining our parameters now ensures that everything is consistent for the remainder of the script.

In [4]:
# Step 0: Define the parameters

NUM_DAYS_HISTORY = 1260      # Approximate number of trading days in five years

NUM_DAYS_FORECAST = 126     # Approximate number of trading days in six months

### **Step 1: Plotting and Accuracy Definitions**

In Python, function definitions create reusable blocks of code that perform specific tasks or represent data structures.

In [5]:
def actual_vs_predicted_plot(actual, predicted, NUM_DAYS_HISTORY, NUM_DAYS_FORECAST):

  """
  The actual_vs_predicted_plot function visualizes a comparison between actual values and predicted values,
  dividing predictions into in-sample (up to NUM_DAYS_HISTORY) and out-of-sample (beyond NUM_DAYS_HISTORY) forecasts.
  """

  # Plot settings
  fig, ax = plt.subplots(figsize=(14, 7))
  plt.title('Actual vs Predicted Values')
  plt.xlabel('')
  plt.ylabel('')


  # Historical prices
  ax.plot(actual, color='black', label='Actual')
  ax.plot(predicted[:NUM_DAYS_HISTORY],
          color='Blue',
          label='Predicted (in-sample)')
  ax.plot(predicted[NUM_DAYS_HISTORY:],
          color='Red',
          label='Predicted (out-of-sample)')

  # plt.legend()
  plt.tight_layout()
  plt

In [6]:
def r_squared_score(actual, predicted):

  """
  The r_squared_score function calculates and prints the R-squared value,
  a statistical measure of how close the data are to the fitted regression line,
  using actual and predicted values.
  """

  r_squared = r2_score(actual, predicted)

  print(f"R-squared value: {r_squared}")

### **Step 2: Get / Generate the data**

Two sets of data are available:


1.   The first data set represents a simple linear relationship (e.g., y = 2x + 1), which is split it into training and test sets.
2.   The second data set comprises market data downloaded from Yahoo Finance.




In [7]:
# Sample Data: Simple linear relationship (for example, y = 2x + 1)

data = {'x': np.arange(0, NUM_DAYS_HISTORY+NUM_DAYS_FORECAST), 'y': (np.arange(0, NUM_DAYS_HISTORY+NUM_DAYS_FORECAST) * 2 + 1)}
df = pd.DataFrame(data)

df_train = df[:NUM_DAYS_HISTORY]
df_test = df[NUM_DAYS_HISTORY:]


In [ ]:
# Market Data

"""
stock = yf.download('GME', period=str(NUM_DAYS_HISTORY+NUM_DAYS_FORECAST+504)+'d', interval='1d')  # SMMT
stock['Ri'] = np.log(stock['Adj Close']/(stock['Adj Close'].shift(1)))
stock['Ri_vol_delta'] = np.log(stock['Volume']/(stock['Volume'].shift(1)))
stock['Ri_spread'] = (stock['High'] - stock['Low'])/stock['Adj Close']

spx = yf.download('^GSPC', period=str(NUM_DAYS_HISTORY+NUM_DAYS_FORECAST+504)+'d', interval='1d')
spx['Rm'] = np.log(spx['Adj Close']/(spx['Adj Close'].shift(1)))
spx['Rm_vol_delta'] = np.log(spx['Volume']/(spx['Volume'].shift(1)))
spx['Rm_spread'] = (spx['High'] - spx['Low'])/spx['Adj Close']

tbill_data = yf.download('^IRX', period=str(NUM_DAYS_HISTORY+NUM_DAYS_FORECAST+504)+'d', interval='1d')
tbill_data['Rf'] = (1+tbill_data['Adj Close'] / 100) ** (1/252) - 1

tbill_data['Rf_spread'] = (tbill_data['High'] - tbill_data['Low'])/stock['Adj Close']

df = pd.merge(stock, spx, left_index=True, right_index=True, how='inner')
df = pd.merge(df, tbill_data, left_index=True, right_index=True, how='inner')

df.dropna(inplace=True)

df = df[['Ri', 'Rm', 'Rf', 'Ri_vol_delta', 'Ri_spread', 'Rm_vol_delta', 'Rm_spread', 'Rf_spread']]

df['y'] = df['Ri'] - df['Rf']
df['x'] = df['Rm'] - df['Rf']

print('\n', df.describe())

df_train = df[-(NUM_DAYS_HISTORY+NUM_DAYS_FORECAST):-NUM_DAYS_FORECAST]
df_test = df[-NUM_DAYS_FORECAST:]
"""

### **The Regression Model**

Our benchmark model is the lineear regression model estimated with OLS.

The model is available from the statsmodels package.

> statsmodels.regression.linear_model.OLS



In [ ]:
indep_var = ['x']

# Assuming df_train['y'] is your dependent variable and df_train['x'] your independent variable
y = df_train['y']

# Add a constant to the model (for the intercept)
X = sm.add_constant(df_train[indep_var])
# X = np.ones(len(df_train['y']))  # This creates an array of ones

# Fit the model
# Make sure the dependent variable (y) is the first argument
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

**Assessing the accuracy of the model**

In [ ]:
y_pred = model.predict(sm.add_constant(df[indep_var]))
y_pred_is = model.predict(sm.add_constant(df_train[indep_var]))
y_pred_oos = model.predict(sm.add_constant(df_test[indep_var]))

actual_vs_predicted_plot(df[['y']], y_pred, NUM_DAYS_HISTORY, NUM_DAYS_FORECAST)

r_squared_score(df[['y']], y_pred)

**Your Turn!**



*   Conduct the regression analysis using only the intercept term; what do we observe?
*   Add a non-linear term to the simulated dataset. Afterwards, (1) execute the linear regression analysis and (2) attempt to include the non-linear aspect into the regression model.



### **The Neural Network Model**

We'll begin by defining a simple neural network with one unit and a linear activation function, using the Keras package:

https://keras.io/guides/sequential_model/

For optimization visualizations, see here:

https://github.com/j-w-yun/optimizer-visualization

In [35]:
# Define the model

def neural_netork(input_dim):
  model = Sequential()

  # Add a single Dense layer (perceptron) with 1 unit, input shape of 1, and a linear activation function
  # Since the linear activation function acts as an identity function, it's equivalent to what you need
  model.add(Dense(1, input_dim=input_dim, activation='linear'))

  # Second hidden layer
  # model.add(Dense(64, activation='relu'))

  # Third hidden layer
  # model.add(Dense(1, activation='linear'))

  # Compile the model
  # The optimizer is set to 'adam', and the loss function to 'mean_squared_error'
  model.compile(optimizer='adam',
                loss='mean_squared_error'
                )

  # Summary of the model to see its structure
  print(model.summary())

  return model

**Training the model**

In [ ]:
# Train the model
# Using df['x'] as input and df['y'] as the target

indep_var = ['x']

model = neural_netork(len(indep_var))
history = model.fit(df_train[indep_var].values, df_train['y'].values,
                    epochs=100,
                    batch_size=10,
                    validation_split=0,
                    verbose=0,
                    shuffle=True
                    )


# Plotting the training (and validation) loss
plt.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()


**Printing the NNs parameters**

In [ ]:
# Print the weights and biases

"""
This code iterates through each layer of a model, retrieves and prints the weights and biases for every layer,
specifying each layer's position in the sequence.
"""

for layer in model.layers:
    weights, biases = layer.get_weights()
    print(f"Weights of layer {model.layers.index(layer)+1}: {weights}")
    print(f"Biases of layer {model.layers.index(layer)+1}: {biases}")

**Assessing the accuracy of the model**

In [ ]:
# Generate predictions
y_pred = pd.Series(model.predict(df[indep_var]).flatten())  # Flatten is used to ensure the prediction array shape matches the target array
y_pred.index = df.index
y_pred_is = pd.Series(model.predict(df_train[indep_var]).flatten())
y_pred_is.index = df_train.index
y_pred_oos = pd.Series(model.predict(df_test[indep_var]).flatten())
y_pred_oos.index = df_test.index

actual_vs_predicted_plot(df[['y']], y_pred, NUM_DAYS_HISTORY, NUM_DAYS_FORECAST)

r_squared_score(df[['y']], y_pred)

**Your Turn!**

With linear and non-linear simulated data:
*   Add a two more layers to the NN; what do we observe?
*   Add a non-linear activation function to the model; what do we observe?


Use the market data to fit and train the regression and NN models, respectively.

*   What do we learn about benefits / disadvantages of each?
*   What do we learn about the behavior of asset prices and financial markets?


